In [ ]:
import pandas as pd
from pathlib import Path
import pylatex as pl
import pymc as pm
import arviz as az
import plotly.express as px
import numpy as np
import pandas as pd

from estival.model import BayesianCompartmentalModel
import estival.priors as esp
import estival.targets as est
from estival.wrappers import pymc as epm
from tbdynamics import model
from tbdynamics.utils import round_sigfig
import plotly.graph_objects as go

## Define variables

In [ ]:
PROJECT_PATH = Path().resolve()
DATA_PATH = PROJECT_PATH / 'data'
pd.options.plotting.backend = "plotly"

time_start = 1800
time_end = 2023
time_step = 0.1

doc_sections = {}
compartments = [
    "susceptible",
    "early_latent",
    "late_latent",
    "infectious",
    "on_treatment",
    "recovered",
]
infectious_compartments = [
    "infectious",
    "on_treatment",
]

age_strata = [0,5,15,35,50,70] 
matrix = np.ones((6, 6)) # Homo mixing

## Define Model

### Base model

In [ ]:
tb_model = model.build_model(
    compartments,
    infectious_compartments,
    age_strata,
    time_start,
    time_end,
    time_step,
    matrix, 
    add_triangular= False 
)

### Params and calibration targets

In [ ]:
params = {
    "start_population_size": 150.000,
}
priors = [
    esp.UniformPrior("start_population_size", (1000000, 3000000)),
]

pop = pd.Series(
    {
        2009: 86025000,
        2019: 96484000,
    
    }
)
targets = [
    est.NormalTarget("total_population", pop, stdev=100000),
]
calibration_model = BayesianCompartmentalModel(tb_model, params, priors, targets)

### Running Calibration

In [ ]:
with pm.Model() as pmc_model:
    start_params = {k: np.clip(v, *calibration_model.priors[k].bounds(0.99)) for k, v in params.items() if k in calibration_model.priors}
    variables = epm.use_model(calibration_model)
    map_params = pm.find_MAP(start=start_params, vars=variables, maxeval= 5000,include_transformed=False)
    map_params = {k: float(v) for k, v in map_params.items()}
    print('Best calibration parameters found:')
for i_param, param in enumerate(map_params):
    print(f'   {param}: {round_sigfig(map_params[param], 4)} (within bound {priors[i_param].bounds()}')

In [ ]:
params.update(map_params)
tb_model.run(params)
derived_df_0 = tb_model.get_derived_outputs_df()

### Population output

In [ ]:
fig2_1 = px.line(
    derived_df_0,
    x=derived_df_0.index,
    y="total_population",
)
fig2_2 = px.scatter(x= pop.index, y = pop.values)
fig2_2.update_traces(marker=dict(color="red"))
fig2_3 = go.Figure(
    data=fig2_1.data + fig2_2.data,
)
fig2_3.update_layout(
    title="Modelled vs Data", title_x=0.5, xaxis_title="Year", yaxis_title="Population"
)
fig2_3.show()

In [ ]:
pop = derived_df_0[[f'total_populationXage_{i}' for i in age_strata]] 
pop.plot.area(title='Modelled populatation by age group')